In [2]:
!pip install flask-ngrok

In [3]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
import pprint
import requests

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/action', methods=['GET', 'POST'])
def process_webhook():
    request_json = request.json
    
    # naver-book 인텐트 중 book 파라미터 데이터 추출
    query = request_json["queryResult"]["parameters"]["book"]

    # 인증 정보
    client_id = ""
    client_secret = ""

    # 기본 url 정보
    url = "https://openapi.naver.com/v1/search/book.json"

    # url 호출 시 전달할 요청 변수 정보
    params = {"query": query,
              "display": 3,
              "sort": "count"}

    # requests 라이브러리를 이용한 책 검색 api 호출
    # get 방식으로 호출(url)/ 요청 변수 전달(params)/ 인증 정보 및 인코딩 정보 전달(header)
    response = requests.get(url=url, params=params,
                            headers={"X-Naver-Client-Id": client_id,
                                     "X-Naver-Client-Secret": client_secret,
                                     "Content-Type": "application/json; charset=utf-8"})
    # 호출 처리 상태 정보 recode 변수에 할당
    rescode = response.status_code

    if (rescode == 200):
        # 호출 처리 상태가 정상(200) 일 경우리턴 받은 책 조회 정보 출력
        pprint.pprint(response.json())
        data = response.json()
    else:
        print("Error Code:", rescode)

    # Naver 책 검색 API 응답 중 실제 책 아이템 데이터 추출 및 출력
    item_list = data["items"]
    pprint.pprint(item_list)

    # Dialogflow로 응답되는 최종 문자열 데이터 구성
    book_list = ""
    for item in item_list:
        book_list += item["title"]
        book_list += " "

    # Dialogflow로 응답되는 최종 데이터 확인
    print(book_list)

    return {"fulfillmentText":book_list}
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7cef943b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Jan/2020 06:10:23] "POST /action HTTP/1.1" 200 -


{'display': 3,
 'items': [{'author': '사이토 고키',
            'description': '직접 구현하고 움직여보며 익히는 가장 쉬운 <b>딥러닝</b> 입문서!『밑바닥부터 시작하는 '
                           '<b>딥러닝</b>』은 라이브러리나 프레임워크에 의존하지 않고, <b>딥러닝</b>의 '
                           '핵심을 ‘밑바닥부터’ 직접 만들어보며 즐겁게 배울 수 있는 본격 <b>딥러닝</b> '
                           '입문서다. 술술 읽힐 만큼 쉽게 설명하였고, 역전파처럼 어려운 내용은 ‘계산... ',
            'discount': '21600',
            'image': 'https://bookthumb-phinf.pstatic.net/cover/114/923/11492334.jpg?type=m1&udate=20191010',
            'isbn': '8968484635 9788968484636',
            'link': 'http://book.naver.com/bookdb/book_detail.php?bid=11492334',
            'price': '24000',
            'pubdate': '20170103',
            'publisher': '한빛미디어',
            'title': '밑바닥부터 시작하는 <b>딥러닝</b> (파이썬으로 익히는 <b>딥러닝</b> 이론과 구현)'},
           {'author': '사이토 고키',
            'description': '직접 구현하면서 배우는 본격 <b>딥러닝</b> 입문서\n'
                           '이번에는 순환 신경망과 자연어 처리다!이 책은 『밑바닥부터 시작하는 '
                           '<b